In [1]:
# !pip install opencv-python

In [2]:
# !pip install facenet-pytorch


In [37]:
%matplotlib auto

Using matplotlib backend: TkAgg


In [38]:
import torch
import cv2
from facenet_pytorch import InceptionResnetV1
import numpy as np
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

def extract_face_encoding(image):
    # Convert the image to RGB format
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Resize the image to (160, 160) which is the input size of the FaceNet model
    image = cv2.resize(image, (160, 160))
    
    # Convert the image to a PyTorch tensor and normalize its values
    image = torch.from_numpy(image.transpose((2, 0, 1))).float().div(255).unsqueeze(0).to(device)
    
    # Extract the facial features from the image using the FaceNet model
    with torch.no_grad():
        face_encoding = resnet(image)[0].cpu().numpy()
    
    return face_encoding

In [39]:
import cv2

# Load the pre-trained Haar Cascade classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def detect_faces(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale image using the Haar Cascade classifier
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Return the coordinates of the detected faces
    return faces


In [40]:
def compare_face_encodings(face_encoding, known_face_encodings):
    return [np.allclose(face_encoding, known_face_encoding, rtol=1e-2, atol=1e-2) for known_face_encoding in known_face_encodings]

In [41]:
def compare(face_encoding, known_face_encodings):
    temp = {}
    for i in range(len(known_face_encodings)):
        dist = np.linalg.norm(face_encoding-known_face_encodings[i])
        temp[i]=dist
    sort_dict = {k: v for k, v in sorted(temp.items(), key=lambda item: item[1])}
    return temp
    

In [48]:
import cv2
import numpy as np
import os

# Load the known faces and their names
known_faces_dir = "known_faces"
known_face_encodings = []
known_face_names = []
for file in os.listdir(known_faces_dir):
    if file.endswith(".jpg") or file.endswith(".png"):
        image = cv2.imread(os.path.join(known_faces_dir, file))
        face_encoding = extract_face_encoding(image)
        known_face_encodings.append(face_encoding)
        known_face_names.append(os.path.splitext(file)[0])

# # Initialize the camera
# cap = cv2.VideoCapture(0)

# while True:
#     # Capture frame-by-frame
#     ret, frame = cap.read()
unknown_faces_dir = "unknown_faces"
file = "abc.jpg"
frame = cv2.imread(os.path.join(unknown_faces_dir, file))
# print(frame)
# cv2.imshow('frame',frame)

# Detect faces in the frame
face_locations = detect_faces(frame)

# Check if any faces are detected
if face_locations is not None:
    print(face_locations)
    # Extract face encodings from the detected faces
    
    face_encodings = []
    for face_location in face_locations:
        face_image = frame[face_location[1]:face_location[1]+face_location[3], face_location[0]:face_location[0]+face_location[2]]
#         cv2.imshow('face_image', face_image)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()
#         cv2.waitKey(1)
        face_encoding = extract_face_encoding(face_image)
        face_encodings.append(face_encoding)

    # Compare face encodings with known faces
    face_names = []
    for face_encoding in face_encodings:
        matches = compare(face_encoding, known_face_encodings)
        print(matches)
        name = "Unknown"
        if any(matches):
            index = matches.index(True)
            name = known_face_names[index]
        face_names.append(name)
    print(face_names)
    # Draw boxes and labels around the detected faces
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
        cv2.rectangle(frame, (left, top), (right, bottom), color, 2)
        cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
else:
    # Display message if no faces are detected
    cv2.putText(frame, "No faces detected", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

# Display the resulting frame
# cv2.imshow('Facial Recognition', frame)
# if cv2.waitKey(0) & 0xFF == ord('q'):
#     break

# Release the camera and close all windows
# cap.release()
cv2.destroyAllWindows()


[[1833 3186   60   60]
 [1753 3209   62   62]
 [ 580 3220   62   62]
 [ 781 1164 1446 1446]
 [2003 3412   52   52]
 [1951 3329   74   74]
 [ 404 3450   59   59]
 [  80 2064   52   52]
 [2048 3101   56   56]]
{0: 1.2250177, 1: 1.4717427, 2: 1.4206684, 3: 1.4838314, 4: 1.5253102, 5: 1.1196952, 6: 1.4408233, 7: 1.4833875, 8: 1.4946399, 9: 1.5348686}


AttributeError: 'dict' object has no attribute 'index'